<a href="https://colab.research.google.com/github/TimoFeyKiselevGA/Language-Classifier/blob/main/JazykovaClasifikace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Part 1, downloading and extracting texts** 

## Imports and

In [ ]:
import xml.etree.ElementTree as etree
import codecs
import csv
import os
import pandas as pd
import pickle
import re
%load_ext google.colab.data_table
import os
import gc
import psutil

pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  del sys.path[0]


## Downloading texts

In [ ]:
languages_to_download = {
  "CZ": True,
  "RU": True,
  "FR": True,
  "DE": True,
  "EN": True,
  "PL": True,
  "IT": True,
  "JA": True,
  "UK": True,
  "AR": True,
  "FI": True,
  "BG": True,
}

languages_links = {
    
    "CZ": "https://dumps.wikimedia.org/cswiki/20210120/cswiki-20210120-pages-articles.xml.bz2",
    "RU": "https://dumps.wikimedia.org/ruwiki/20210101/ruwiki-20210101-pages-articles-multistream5.xml-p3835773p5335772.bz2",
    "FR": "https://dumps.wikimedia.org/frwiki/20210101/frwiki-20210101-pages-articles-multistream2.xml-p306135p1050822.bz2",
    "DE": "https://dumps.wikimedia.org/dewiki/20210101/dewiki-20210101-pages-articles-multistream6.xml-p9261245p10761244.bz2",
    "EN": "https://dumps.wikimedia.org/enwiki/20210101/enwiki-20210101-pages-articles-multistream9.xml-p2936261p4045402.bz2",
    "PL": "https://dumps.wikimedia.org/plwiki/20210101/plwiki-20210101-pages-articles-multistream4.xml-p1199519p2047892.bz2",
    "IT": "https://dumps.wikimedia.org/itwiki/20210101/itwiki-20210101-pages-articles-multistream4.xml-p2206775p3593336.bz2",
    "JA": "https://dumps.wikimedia.org/jawiki/20210101/jawiki-20210101-pages-articles-multistream5.xml-p1721647p2807947.bz2",
    "UK": "https://dumps.wikimedia.org/ukwiki/20210120/ukwiki-20210120-pages-articles-multistream4.xml-p987986p1674442.bz2",
    "AR": "https://dumps.wikimedia.org/arwiki/20210120/arwiki-20210120-pages-articles-multistream4.xml-p2482316p3982315.bz2",
    "FI": "https://dumps.wikimedia.org/fiwiki/20210120/fiwiki-20210120-pages-articles-multistream.xml.bz2",
    "BG": "https://dumps.wikimedia.org/bgwiki/20210120/bgwiki-20210120-pages-articles-multistream.xml.bz2",
}

languages = ['CZ', 'RU', 'FR', 'DE', 'EN', 'PL', 'IT', 'JA', "UK", "AR", "FI", "BG"]
languages_to_process = {
  "CZ": True,
  "RU": True,
  "FR": True,
  "DE": True,
  "EN": True,
  "PL": True,
  "IT": True,
  "JA": True,
  "UK": True,
  "AR": True,
  "FI": True,
  "BG": True,
}

In [ ]:
def rename(s):
    return languages_links[s].split("/")[len(languages_links[s].split("/"))-1].split(".")[0]+ "." +languages_links[s].split("/")[len(languages_links[s].split("/"))-1].split(".")[1]

In [ ]:
for lang in languages:
    if languages_to_download[lang]:
        print("downloading " + lang)
        os.system("wget " + languages_links[lang])
        print("extracting " + lang)
        os.system("bzip2 -d ./ " + languages_links[lang].split("/")[len(languages_links[lang].split("/"))-1])
        os.system("mv " + rename(lang) + " " + languages_links[lang].split("/")[len(languages_links[lang].split("/"))-1].split(".")[0] + ".xml")


downloading CZ
extracting CZ
downloading RU
extracting RU
downloading FR
extracting FR
downloading DE
extracting DE
downloading EN
extracting EN
downloading PL
extracting PL
downloading IT
extracting IT
downloading JA
extracting JA
downloading UK
extracting UK
downloading AR
extracting AR
downloading FI
extracting FI
downloading BG
extracting BG


## Creating pkl files with texts without most special chars. from xmls.

In [ ]:
def strip_tag_name(t):
    idx = k = t.rfind("}")
    if idx != -1:
        t = t[idx + 1:]
    return t

In [ ]:
def articles_to_csv(totalCount):
    with codecs.open(pathTexts, "w", ENCODING) as textsFH:
        textWriter = csv.writer(textsFH, quoting=csv.QUOTE_MINIMAL)
        textWriter.writerow(['text'])
        for event, elem in etree.iterparse(pathWikiXML, events=('start', 'end')):
            tname = strip_tag_name(elem.tag)
            if event == 'start':
                if tname == 'page':
                    title = ''
                    id = -1
                    redirect = ''
                    inrevision = False
                    ns = 0
                    text = ''
                elif tname == 'text':
                    text = elem.text
                    textWriter.writerow([text])
                elif tname == 'page':
                    totalCount += 1
                if totalCount > 1 and (totalCount % 100000) == 0:
                    print("{:,}".format(totalCount))
            elem.clear()

In [ ]:
for lang in languages:
    if languages_to_process[lang]:
        print(lang)
        PATH_WIKI_XML = './'
        FILENAME_WIKI = languages_links[lang].split("/")[len(languages_links[lang].split("/"))-1].split('.')[0] + '.xml'
        texts_template = 'temp.csv'
        ENCODING = "utf-8"
        pathWikiXML = os.path.join(PATH_WIKI_XML, FILENAME_WIKI)
        pathTexts = os.path.join(PATH_WIKI_XML, texts_template)
        totalCount = 0
        articleCount = 0
        title = None
        articles_to_csv(totalCount)
        a = pd.read_csv('temp.csv')
        a = a.dropna()
        a['text'] = a['text'].str.replace(',|\$|\{|\}|\-|\+|\[|\]|\\n|\<|\>|\"|\=|\:|\;|\*|\||\#|\$|\(|\)|\/|\.|\–|\_|\«|\»|\—|\!|\&|\?|\`|\~|\%|\@|\¡|\™|\£|\¢|\∞|\§|\¶|\•|\ª|\º|\–|\≠|\«|\æ|\÷|\≥|\≤|\÷|\»|\\|\^|\ˆ|\|\\|\||\）|\（|\、|\'', ' ')
        a['text'] = a['text'].str.replace('\d+', '')
        a['text'] = a['text'].str.replace('\s\s+' , ' ')
        a = a.drop([a.index[0]])
        a = a.reset_index(drop=True)
        a['text'] = a['text'].str.lower()
        a.to_pickle("./texts_" + lang +  ".pkl")
        

CZ
RU
FR
DE
EN
PL
IT
JA
UK
AR
FI
BG


In [ ]:
%reset -f

# **Part 2: Dataset preparation**

## Imports and settings

In [ ]:
import xml.etree.ElementTree as etree
import codecs
import csv
import os
import pandas as pd
import pickle
import re
%load_ext google.colab.data_table
import os
import gc
import psutil
import numpy as np
import random
#random seeds
random.seed(0)
np.random.seed(0)

pd.set_option('display.max_colwidth', -1)

languages = ['CZ', 'RU', 'FR', 'DE', 'EN', 'PL', 'IT', 'JA', "UK", "AR", "FI", "BG"]
languages_to_process = {
  "CZ": True,
  "RU": True,
  "FR": True,
  "DE": True,
  "EN": True,
  "PL": True,
  "IT": True,
  "JA": True,
  "UK": True,
  "AR": True,
  "FI": True,
  "BG": True,
}

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


## Dataset creation

In [ ]:
def chunkstring(string, length):
    return re.findall('.{%d}' % length, string)

def toPickleLoadList(string, textList):
    file_name = "texts_" + string + "_prep.pkl"
    open_file = open(file_name, "wb")
    pickle.dump(textList, open_file)
    open_file.close()
    print("done " + lang)

def fromPickleLoadList(string):
    file_name = "texts_" + string + "_prep.pkl"
    open_file = open(file_name, "rb")
    loaded_list = pickle.load(open_file)
    open_file.close()
    return loaded_list

def chunkDataFrames(lang):
    print("start " + lang)
    df = pd.read_pickle("./texts_"+ lang + ".pkl")
    df = df[df['text'].apply(lambda x: len(x)>100)]
    # df = df[~df.text.str.startswith(' redirect', na=True)]
    df.text = df.text.replace('\s+', ' ', regex=True)
    df = df.text.str.cat(sep='')
    textList = chunkstring(df, 200)
    toPickleLoadList(lang, textList)
    del df;  gc.collect()
    del textList;  gc.collect()


In [ ]:
for lang in languages:
    if languages_to_process[lang]:
        chunkDataFrames(lang)
    

start CZ
done CZ
start RU
done RU
start FR
done FR
start DE
done DE
start EN
done EN
start PL
done PL
start IT
done IT
start JA
done JA
start UK
done UK
start AR
done AR
start FI
done FI
start BG
done BG


In [ ]:
from sklearn.utils import shuffle
text_to_tokens = []
id_lang = []
id = 0
delka = 100000

for lang in languages:
    df = shuffle(fromPickleLoadList(lang))
    df = df[:delka]
    id_lang = id_lang + [id] * delka
    id = id + 1
    text_to_tokens = text_to_tokens + df

In [ ]:
data_tuples = list(zip(text_to_tokens,id_lang))
df = pd.DataFrame(data_tuples, columns=['X','Y'])

In [ ]:
df = shuffle(df)

In [ ]:
df.head()

,X,Y
843521,овного короля ірландії іререо ірл nbsp irereo що був вбивцею його батька це сталося на території васального королівства улад ольстер правив ірландією протягом одинадцяти років був вбитий сином іререо,8
882105,bsp українське прізвище яке має комплексну етимологію воно походить від кількох імен на зразок krisant kristofor kristijan крискент кресимир kpecoje кирсантій також є прізвища від польських основ kres,8
192632,зыка м фасмера возводит происхождение слова плёс к древнерусскому слову плесъ глубокое место в воде озере семантически наиболее вероятно родство со славянскими корнями рlesо рletsо греч πλάτος произво,1
80787,usindia net results town php stad a state datum archivace nedostupné ne ref reference references externí odkazy commonscat itanagar indie portály indie autoritní data kategorie geografie arunáčalpradé,0
844187,адемічній праці грамматика современного русского литературного языка року пояснюються основні терміни звуженою парадигмою називається парадигма репрезентована синтетичними формами які виражають основн,8


# Part 3: Model training

## Imports and settings


In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import torch
from torch import nn
from torch.nn import functional as F
from sklearn.metrics import accuracy_score

## Model creation and training

In [ ]:
def simple_tokenizer(txt, min_len=2):
    all_tokens = re.compile(r'[\w\d]+').findall(txt)
    all_tokens = [x.lower() for x in all_tokens]
    return [token for token in all_tokens if len(token) >= min_len]

In [ ]:
sklearn_pipeline = Pipeline((('vect', TfidfVectorizer(tokenizer=simple_tokenizer,
                                                      max_df=0.8,
                                                      min_df=50)),
                             ('cls', LogisticRegression())))

In [ ]:
sklearn_pipeline.fit(train['X'].to_numpy(), train['Y'].to_numpy())

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.8, max_features=None,
                                 min_df=50, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...w+\\b',
                                 tokenizer=<function simple_tokenizer at 0x7f6f8c35e1e0>,
                                 use_idf=True, vocabulary=None)),
                ('cls',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_i

In [ ]:
sklearn_train_pred = sklearn_pipeline.predict_proba(train['X'].to_numpy())
sklearn_train_loss = F.cross_entropy(torch.from_numpy(sklearn_train_pred),
                                                 torch.from_numpy(train['Y'].to_numpy()))

In [ ]:
print('train loss', float(sklearn_train_loss))
print('acc', accuracy_score(train['Y'], sklearn_train_pred.argmax(-1)))
print()

train loss 1.688278924455967
acc 0.969027380952381



In [ ]:
sklearn_test_pred = sklearn_pipeline.predict_proba(test['X'].to_numpy())
sklearn_test_loss = F.cross_entropy(torch.from_numpy(sklearn_test_pred),
                                                torch.from_numpy(test['Y'].to_numpy()))

In [ ]:
print('Val loss', float(sklearn_test_loss))
print('acc', accuracy_score(test['Y'], sklearn_test_pred.argmax(-1)))

NameError: ignored

In [ ]:
pokus = ['hilft zu lernen oder.', 'no jo ale to neni  ', ' s a song by the', 'Существуют оба потом']
for i in range(0, len(pokus)):
    print(languages[sklearn_pipeline.predict_proba(pokus).argmax(-1)[i]])

NameError: ignored

In [ ]:
languages = ['CZ', 'RU', 'FR', 'DE', 'EN', 'PL', 'IT', 'JA', "UK", "AR", "FI", "BG"]

In [ ]:
pickle.dump(sklearn_pipeline, open("model.pkl", 'wb'))

In [ ]:
loaded_model = pickle.load(open("model1.pkl", 'rb'))

In [ ]:
pokus = ['hilft zu lernen oder.', 'njn jak nn se mas ', ' s a song by the',
         'Существуют оба потом', 'Южнославянски език, написан на кирилица', 'ファベットブルガリア語キリル文字',
         'slaavilainen kieli, joka', 'اللغة السلافية الجنوبية مكتوبة']
for i in range(0, len(pokus)):
    print(loaded_model.predict_proba(pokus)[i])
    print(languages[loaded_model.predict_proba(pokus).argmax(-1)[i]])

[0.01040848 0.01026844 0.00402816 0.7843419  0.002695   0.01620735
 0.00559595 0.02351029 0.00924845 0.00482381 0.00927008 0.11960209]
DE
[0.48002282 0.02351725 0.0375638  0.02413878 0.01949789 0.135937
 0.04214942 0.04339878 0.01913916 0.01364655 0.13951835 0.02147019]
CZ
[4.57926015e-03 1.03126948e-02 9.88920814e-04 7.94266947e-04
 9.33383964e-01 5.37864381e-03 2.51477789e-03 2.97311970e-02
 4.25743902e-03 2.72081517e-03 2.07491178e-03 3.26310854e-03]
EN
[0.05905843 0.25698127 0.04660967 0.03576053 0.03032131 0.08022452
 0.04794622 0.25286381 0.05001107 0.03405699 0.0741988  0.03196736]
RU
[1.40725206e-05 1.57515026e-03 1.17495153e-05 7.75252116e-06
 5.57067940e-06 2.04064643e-05 1.29130646e-05 2.53488522e-05
 1.87944820e-03 1.01154519e-05 1.68807970e-05 9.96420592e-01]
BG
[0.07016339 0.08005213 0.05307346 0.04131687 0.03641797 0.09590707
 0.05529808 0.32347421 0.06249334 0.04089406 0.08757327 0.05333615]
JA
[1.01512069e-05 1.17832362e-05 8.16780927e-06 6.24424098e-06
 4.79296175e-06

## Save links for future documentation

In [ ]:
#https://github.com/jeffheaton/present/blob/master/youtube/read_wikipedia.ipynb
# https://stackoverflow.com/questions/18854620/whats-the-best-way-to-split-a-string-into-fixed-length-chunks-and-work-with-the/18854817